In [1]:
from flask import Flask, request, jsonify,Request

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import folium
from geopy.geocoders import Nominatim
from flask_cors import CORS
from threading import Thread
from waitress import serve  # Production server
from flask import Response
from io import BytesIO

In [2]:
df = pd.read_csv("Crime_Dataset.csv")

# One-hot encode the Region column
encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(df[['Region']])

# Normalize the crime data
scaler = StandardScaler()
y_scaled = scaler.fit_transform(df.drop(columns=['Region']))

# Load trained model
model = keras.models.load_model("crime_prediction_model.h5", custom_objects={"mse": keras.losses.MeanSquaredError()})

# Compile the model to avoid warning
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [3]:
precautions_dict = {
    "Abandoning Infant": ["Report abandoned infants to authorities.", "Ensure child safety measures.", "Support child welfare programs."],
    "Abetment of Suicide": ["Encourage mental health support.", "Report coercion or threats.", "Promote awareness about suicide prevention."],
    "Acid Attack": ["Be aware of surroundings.", "Avoid conflicts with hostile individuals.", "Report any threats immediately."],
    "Arson": ["Ensure fire safety measures.", "Report suspicious activities.", "Avoid storing flammable substances openly."],
    "Assault on Women with intent to outrage her Modesty": ["Avoid isolated locations.", "Use personal safety apps.", "Report incidents immediately."],
    "Attempt to Acid Attack": ["Stay vigilant in public spaces.", "Report threats to authorities.", "Educate about acid attack laws."],
    "Attempt to commit Culpable Homicide": ["Stay away from hostile situations.", "Alert security personnel if threatened.", "Avoid interactions with violent individuals."],
    "Attempt to commit Murder": ["Avoid suspicious individuals.", "Report threats to authorities.", "Stay in secure locations."],
    "Attempt to commit Rape": ["Avoid secluded areas.", "Use emergency contacts.", "Report harassment immediately."],
    "Attempt to commit Suicide": ["Seek mental health assistance.", "Encourage counseling services.", "Provide emotional support."],
    "Auto Theft": ["Use anti-theft devices.", "Park in secure locations.", "Avoid leaving valuables inside vehicles."],
    "Buying or Selling of Minors for prostitution": ["Report human trafficking activities.", "Educate about child protection laws.", "Support anti-trafficking initiatives."],
    "Causing hurt by rash driving": ["Follow traffic rules.", "Avoid overspeeding.", "Stay alert while driving."],
    "Cheating": ["Verify before making financial transactions.", "Avoid sharing personal details.", "Be skeptical of unrealistic offers."],
    "Child Abuse": ["Report suspected child abuse immediately.", "Educate children about personal safety.", "Support child protection agencies."],
    "Child Marriage": ["Report underage marriages.", "Promote awareness of legal marriage age.", "Support victims of forced marriages."],
    "Child Pornography": ["Monitor children's internet usage.", "Report online child exploitation.", "Educate about online safety."],
    "Counterfeiting": ["Verify authenticity of currency and goods.", "Report counterfeit items.", "Avoid suspicious transactions."],
    "Cyber Stalking": ["Do not share personal information online.", "Block and report suspicious users.", "Enable privacy settings on social media."],
    "Identity Theft": ["Use multi-factor authentication.", "Monitor bank statements regularly.", "Report unauthorized transactions."],
    "Hate Crimes": ["Report hate speech and violence.", "Support victims of hate crimes.", "Promote community harmony."],
    "Bribery": ["Refuse to participate in bribery.", "Report corruption cases.", "Support ethical governance."],
    "Human Trafficking": ["Report suspicious activities.", "Avoid illegal employment agencies.", "Support anti-trafficking organizations."],
    "Extortion": ["Do not respond to blackmail threats.", "Report extortion attempts to authorities.", "Secure important personal and business data."],
    "Insult to Modesty of Women": ["Call out inappropriate behavior.", "Use legal channels for complaints.", "Support awareness campaigns."],
    "Forced Labor": ["Report exploitation of workers.", "Encourage fair labor policies.", "Support ethical businesses."],
    "Public Nuisance": ["Respect public spaces.", "Report disturbances to authorities.", "Encourage civic responsibility."],
    "Sexual Exploitation": ["Educate about consent and rights.", "Support victims with legal aid.", "Report perpetrators to authorities."],
    "Theft": ["Keep valuables secure.", "Avoid displaying expensive items in public.", "Report theft immediately."],
    "Terrorism": ["Stay aware of surroundings.", "Report suspicious activities.", "Follow emergency protocols."],
    "Vandalism": ["Secure property with surveillance.", "Report acts of vandalism.", "Promote community awareness."],
    "Voyeurism": ["Report hidden cameras in public spaces.", "Be cautious in private settings.", "Support strict privacy laws."],
    "Misuse of Social Media": ["Verify information before sharing.", "Report online abuse and misinformation.", "Use responsible social media practices."]
}

In [4]:
def get_coordinates(region):
    """
    Get the geographical coordinates (latitude and longitude) for a given region.
    """
    geolocator = Nominatim(user_agent="crime_map")
    location = geolocator.geocode(region, timeout=10)
    return (location.latitude, location.longitude) if location else None

def generate_crime_map(coordinates, top_crimes, precautions_output):
    """
    Generate a crime map and return the HTML as a response instead of saving a file.
    """
    crime_map = folium.Map(location=coordinates, zoom_start=10)
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'pink', 'brown', 'gray', 'black']

    for i, (crime, severity) in enumerate(top_crimes.items()):
        precautions_list = precautions_output.get(crime, ["No specific precautions available."])
        precautions_text = "<br>".join(precautions_list)
        
        popup_text = f"""
        <b>{crime}</b>: {severity:.2f} (Severity Level) <br>
        <b>Precautions:</b> <br> {precautions_text}
        """
        popup = folium.Popup(popup_text, max_width=300)

        folium.CircleMarker(
            location=[coordinates[0] + (0.01 * i), coordinates[1] + (0.01 * i)],
            radius=7,
            color=colors[i % len(colors)],
            fill=True,
            fill_color=colors[i % len(colors)],
            fill_opacity=0.6,
            popup=popup
        ).add_to(crime_map)

    # Convert map to HTML string
    return crime_map._repr_html_()


In [5]:
app = Flask(__name__)
CORS(app)  # Enable CORS

@app.route('/predict', methods=['POST'])
def predict_crime():
    """
    API Endpoint to predict crimes and return a map URL.
    """
    data = request.json
    region = data.get("region")

    if not region:
        return jsonify({"error": "Region not provided"}), 400

    coordinates = get_coordinates(region)
    if not coordinates:
        return jsonify({"error": "Location not found"}), 404

    # Transform input for prediction
    region_encoded = encoder.transform(pd.DataFrame([[region]], columns=['Region']))
    predictions = model.predict(region_encoded)
    predicted_crimes = scaler.inverse_transform(predictions)
    crime_data = pd.DataFrame(predicted_crimes, columns=df.drop(columns=['Region']).columns)

    # Top 10 crimes
    top_crimes = crime_data.iloc[0].nlargest(10)

    # Precautions
    precautions_output = {
        crime: precautions_dict.get(crime, ["No specific precautions available."])
        for crime in top_crimes.index
    }

    # Generate map HTML
    crime_map_html = generate_crime_map(coordinates, top_crimes, precautions_output)

    # Save map to file
    with open("crime_map.html", "w", encoding="utf-8") as f:
        f.write(crime_map_html)

    # Return map URL (served via Flask)
    return jsonify({"map_url": "http://127.0.0.1:5001/crime_map.html"})


@app.route('/crime_map.html')
def serve_map():
    """Serve the saved crime map."""
    return open("crime_map.html", encoding="utf-8").read()


def run_flask():
    """Run the Flask app on a different port (not 3000)."""
    serve(app, host="127.0.0.1", port=5001)


# If running in Jupyter notebook or as script
if __name__ == '__main__':
    Thread(target=run_flask).start()